In [284]:
import pandas as pd
import os
import os.path as osp
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob

In [285]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

In [286]:
# path_to_annotations = "/videos/mpi_data/2Itzik/dyadic_communication/annotations.csv"
path_to_annotations = "/videos/mpi_data/2Itzik/dyadic_communication/ANNOTATIONS/20240125_annotations.csv"
df_raw = pd.read_csv(path_to_annotations)
df_raw.drop('Unnamed: 0',axis=1, inplace=True)

In [287]:
# Recode shrg as shrai 
# Rename hands/arms as hands_arms
df_raw['label'] = df_raw['label'].replace('shrg', 'shrai')
df_raw['tier'] = df_raw['tier'].replace('hands/arms', 'hands_arms')

In [288]:
# remove rows from annotation table if we don't have relevant videos
before_participants = df_raw["participant_id"].unique()
print(f'Before removal: {before_participants}')
participants = [
    'ID00',
    'ID11',
    'ID02',
    'ID03',
    'ID04',
    'ID05',
    'ID06',
    'ID07',
    # 'ID08'
]

cameras = [
    'Cam3',
    'Cam4'
]

seats = [
    'left', 'right'
]
df_raw = df_raw.loc[df_raw['participant_id'].isin(participants)]


after_participants = df_raw["participant_id"].unique()
print(f'After removal: {df_raw["participant_id"].unique()}')
print(f'Removed rows for participant/s: {set(before_participants)-set(after_participants)}')
# recode participant_id to match file names
df_raw["participant_id"] = df_raw["participant_id"].apply(lambda x: '_'.join(['PIS',x[:2],x[2:]]))

df_raw = df_raw.loc[df_raw['seat'].isin(seats)]

Before removal: ['ID00' 'ID11' 'ID02' 'ID03' 'ID04' 'ID05' 'ID06' 'ID07' 'ID08']
After removal: ['ID00' 'ID11' 'ID02' 'ID03' 'ID04' 'ID05' 'ID06' 'ID07']
Removed rows for participant/s: {'ID08'}


In [294]:
df_raw_dummy.columns

Index(['begin', 'end', 'participant_id', 'seat', 'tier', 'blean', 'flean',
       'fmov', 'hmov', 'hnod', 'hrep', 'hshk', 'htfa', 'hthd', 'lgcr', 'lgucr',
       'readj', 'shrai'],
      dtype='object')

In [243]:
# df_raw["participant_id"].unique()

In [233]:
combinations_df = df_raw.groupby(['tier', 'label']).size().reset_index(name='count')

combinations_df

,tier,label,count
0,fubo,readj,159
1,hands_arms,hrep,1400
2,hands_arms,htfa,364
3,hands_arms,hthd,115
4,head,hmov,1046
5,head,hnod,1786
6,head,hshk,616
7,lobo,fmov,1837
8,lobo,lgcr,41
9,lobo,lgucr,49


In [234]:
feature_names = df_raw['label'].unique()
tier_names = df_raw['tier'].unique()

In [235]:
len(df_raw.loc[df_raw['label']=='hmov'])

1046

In [300]:
# Create a new column indicating the presence of a label
# Convert 'label' column into one-hot encoded columns
df_raw_dummy = pd.get_dummies(df_raw, columns=['label'])
df_raw_dummy.columns = df_raw_dummy.columns.str.replace('label_', '')


# Group by other columns and aggregate using max function
reshaped_df = df_raw_dummy.drop(columns='tier').groupby(['begin', 'end', 'participant_id', 'seat']).max().astype(int).reset_index()


# Pivot the table
# reshaped_df = df_raw.groupby(['participant_id','begin', 'end', 'seat', 'label'])['label_presence'].max().unstack(level=['label'], fill_value=0)

# Reset index to make 'begin', 'end' as columns
reshaped_df = reshaped_df.reset_index(drop=True)

# Add this line after reshaped_df = reshaped_df.reset_index()
reshaped_df.columns = [f'{col[0]}_{col[1]}' if isinstance(col, tuple) else col for col in reshaped_df.columns]

# Remove training _
reshaped_df.columns = [col[:-1] if col.endswith('_') else col for col in reshaped_df.columns]

# Remove end time and make begin the index
# reshaped_df = reshaped_df.drop('end', axis=1).set_index(['begin','seat'])
reshaped_df = reshaped_df.drop('end', axis=1).set_index(['participant_id','begin','seat'])
reshaped_df.head(1000)

,,,blean,flean,fmov,hmov,hnod,hrep,hshk,htfa,hthd,lgcr,lgucr,readj,shrai
participant_id,begin,seat,,,,,,,,,,,,,
PIS_ID_00,0,left,0,0,1,0,0,0,0,0,0,0,0,0,0
PIS_ID_03,0,left,0,0,1,0,1,1,0,0,0,0,0,0,0
PIS_ID_06,0,left,0,0,1,0,1,0,0,0,0,0,0,0,0
PIS_ID_07,0,left,0,0,0,1,0,0,0,0,0,0,0,0,0
PIS_ID_11,0,right,1,1,1,1,1,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PIS_ID_04,390000,left,0,0,0,0,0,1,0,0,0,0,0,0,0
PIS_ID_05,390000,left,0,0,0,0,1,1,0,0,0,0,0,0,0
PIS_ID_06,390000,left,0,0,1,0,1,1,0,0,0,0,0,0,0


In [301]:
reshaped_df['hnod'].unique()

array([0, 1])

In [302]:
# add chagnes to the labels 
# 1. merge htfa into hthd
# reshaped_df['htfa'] = reshaped_df['htfa'] | reshaped_df['hthd']
# reshaped_df.drop(columns='hthd', inplace=True)  # Drop 'hthd'

# 2. remove lgcr and lgucr
try:
    reshaped_df.drop(columns='lgcr', inplace=True)  # Drop 'hthd'
    reshaped_df.drop(columns='lgucr', inplace=True)  # Drop 'hthd'
except:
    pass

#3. add to hmov all classes of head movment - hnod, hshk
reshaped_df['hmov'] = reshaped_df['hnod'] | reshaped_df['hshk'] | reshaped_df['hmov']

#4. add to readj all readjustments - blean, flean
reshaped_df['readj'] = reshaped_df['readj'] | reshaped_df['blean'] | reshaped_df['flean']

feature_names = reshaped_df.columns

In [303]:
len(reshaped_df.loc[reshaped_df['hmov']==1])

2525

In [209]:
# # Pivot the table
# reshaped_df = df_raw.groupby(['begin', 'end', 'seat', 'tier'])['label_presence'].max().unstack(level=['tier'], fill_value=0)

# # Reset index to make 'begin', 'end' as columns
# reshaped_df = reshaped_df.reset_index()

# # Add this line after reshaped_df = reshaped_df.reset_index()
# reshaped_df.columns = [f'{col[0]}_{col[1]}' if isinstance(col, tuple) else col for col in reshaped_df.columns]

# # Remove training _
# reshaped_df.columns = [col[:-1] if col.endswith('_') else col for col in reshaped_df.columns]

# # Remove end time and make begin the index
# reshaped_df = reshaped_df.drop('end', axis=1).set_index(['begin','seat'])
# reshaped_df.head()

In [210]:
# path_to_videos = '/videos/mpi_data/2Itzik/dyadic_communication/PIS_ID_000_SPLIT/*/*/'
path_to_videos = '/videos/mpi_data/2Itzik/dyadic_communication/SPLIT/*/*/'
file_folders = glob.glob(path_to_videos, recursive = False)


In [197]:
def get_time_from_file_name(file_path, segment_length = 2000):
    base_name = os.path.basename(file_path)
    file_name, _ = os.path.splitext(base_name)

    segmment_time = int(''.join(filter(str.isdigit, file_name))) * segment_length
    return segmment_time

def get_participant_id_from_file_name(file_path):
    return file_path.split('/')[-4]

In [304]:
# path_to_videos = '/videos/mpi_data/2Itzik/dyadic_communication/PIS_ID_000_SPLIT/'
path_to_videos = '/videos/mpi_data/2Itzik/dyadic_communication/SPLIT/'
all_video_files = glob.glob(osp.join(path_to_videos,'*','*','*','*.mp4'))

# Remove head view and only keep top and front view
all_video_files = [f for f in all_video_files if 'Cam3' in f or 'Cam4' in f]

df_files = pd.DataFrame(all_video_files, columns=['filenames'])
df_files['view'] = ['top' if 'Cam3' in c else 'front' for c in all_video_files]
df_files['seat'] = ['left' if 'left' in c else 'right' for c in all_video_files]
df_files['begin'] = df_files['filenames'].apply(get_time_from_file_name)
df_files['participant_id'] = df_files['filenames'].apply(get_participant_id_from_file_name)

df_files = df_files.loc[df_files['seat'].isin(seats)]
df_files = df_files.loc[df_files['participant_id'].isin(reshaped_df.reset_index()['participant_id'].unique())]
df_files = df_files.set_index(['participant_id','begin','seat'])
df_files.head()


filenames  \
participant_id begin seat                                                      
PIS_ID_00      0     left  /videos/mpi_data/2Itzik/dyadic_communication/S...   
               2000  left  /videos/mpi_data/2Itzik/dyadic_communication/S...   
               4000  left  /videos/mpi_data/2Itzik/dyadic_communication/S...   
               6000  left  /videos/mpi_data/2Itzik/dyadic_communication/S...   
               8000  left  /videos/mpi_data/2Itzik/dyadic_communication/S...   

                          view  
participant_id begin seat       
PIS_ID_00      0     left  top  
               2000  left  top  
               4000  left  top  
               6000  left  top  
               8000  left  top

In [305]:
# df = df_files.join(reshaped_df).fillna(0)
# df.sum()

In [306]:
df = df_files.join(reshaped_df).fillna(0)
df['mov'] = (df[feature_names] == 1).any(axis=1)
df['bg'] = (df[feature_names] == 0).all(axis=1)
df = df.reset_index().set_index(['participant_id','begin','seat','filenames','view'], drop=True)
feature_names = df.columns.to_list()
# df = df.reset_index().set_index(['participant_id','view','seat','begin','filenames'])
# feature_names = df.columns
# df = df.astype(int)
# df = df.reset_index().set_index(['participant_id','view','seat','begin'])
df.head()

blean  \
participant_id begin seat  filenames                                          view           
PIS_ID_00      0     left  /videos/mpi_data/2Itzik/dyadic_communication/SP... top      0.0   
                           /videos/mpi_data/2Itzik/dyadic_communication/SP... front    0.0   
                     right /videos/mpi_data/2Itzik/dyadic_communication/SP... top      0.0   
                           /videos/mpi_data/2Itzik/dyadic_communication/SP... front    0.0   
               2000  left  /videos/mpi_data/2Itzik/dyadic_communication/SP... top      0.0   

                                                                                     flean  \
participant_id begin seat  filenames                                          view           
PIS_ID_00      0     left  /videos/mpi_data/2Itzik/dyadic_communication/SP... top      0.0   
                           /videos/mpi_data/2Itzik/dyadic_communication/SP... front    0.0   
                     right /videos/mpi_data/2Itzik/dyadic_communication/SP... top      0.0   
                           /videos/mpi_data/2Itzik/dyadic_communication/SP... front    0.0   
               2000  left  /videos/mpi_data/2Itzik/dyadic_communication/SP... top      0.0   

                                                                                     fmov  \
participant_id begin seat  filenames                                          view          
PIS_ID_00      0     left  /videos/mpi_data/2Itzik/dyadic_communication/SP... top     1.0   
                           /videos/mpi_data/2Itzik/dyadic_communication/SP... front   1.0   
                     right /videos/mpi_data/2Itzik/dyadic_communication/SP... top     0.0   
                           /videos/mpi_data/2Itzik/dyadic_communication/SP... front   0.0   
               2000  left  /videos/mpi_data/2Itzik/dyadic_communication/SP... top     1.0   

                                                                                     hmov  \
participant_id begin seat  filenames                                          view          
PIS_ID_00      0     left  /videos/mpi_data/2Itzik/dyadic_communication/SP... top     0.0   
                           /videos/mpi_data/2Itzik/dyadic_communication/SP... front   0.0   
                     right /videos/mpi_data/2Itzik/dyadic_communication/SP... top     0.0   
                           /videos/mpi_data/2Itzik/dyadic_communication/SP... front   0.0   
               2000  left  /videos/mpi_data/2Itzik/dyadic_communication/SP... top     0.0   

                                                                                     hnod  \
participant_id begin seat  filenames                                          view          
PIS_ID_00      0     left  /videos/mpi_data/2Itzik/dyadic_communication/SP... top     0.0   
                           /videos/mpi_data/2Itzik/dyadic_communication/SP... front   0.0   
                     right /videos/mpi_data/2Itzik/dyadic_communication/SP... top     0.0   
                           /videos/mpi_data/2Itzik/dyadic_communication/SP... front   0.0   
               2000  left  /videos/mpi_data/2Itzik/dyadic_communication/SP... top     0.0   

                                                                                     hrep  \
participant_id begin seat  filenames                                          view          
PIS_ID_00      0     left  /videos/mpi_data/2Itzik/dyadic_communication/SP... top     0.0   
                           /videos/mpi_data/2Itzik/dyadic_communication/SP... front   0.0   
                     right /videos/mpi_data/2Itzik/dyadic_communication/SP... top     0.0   
                           /videos/mpi_data/2Itzik/dyadic_communication/SP... front   0.0   
               2000  left  /videos/mpi_data/2Itzik/dyadic_communication/SP... top     0.0   

                                                                                     hshk  \
participant_id begin seat  filenames                              

In [307]:
df = df.reset_index()
df['participant_id_seat'] = (df.reset_index()['participant_id'] +'_'+ df.reset_index()['seat']).values
# df['participant_id_seat']

In [308]:
df.columns

Index(['participant_id', 'begin', 'seat', 'filenames', 'view', 'blean',
       'flean', 'fmov', 'hmov', 'hnod', 'hrep', 'hshk', 'htfa', 'hthd',
       'readj', 'shrai', 'mov', 'bg', 'participant_id_seat'],
      dtype='object')

In [309]:
# I want all the 'right' files to be in the training data!
df_with_right = df.loc[df['seat']=='left']

# X = df['filenames'].to_numpy()
X = df_with_right['filenames'].to_numpy()
# df['participant_id_dummy'] = df['participant_id'].astype('category').cat.codes
# df['seat_dummy'] = df['seat'].astype('category').cat.codes

all_features = list(feature_names)+['participant_id','seat']
# y = df[all_features].to_numpy()
y = df_with_right[all_features].to_numpy()

msss = MultilabelStratifiedShuffleSplit(n_splits=2, test_size=0.5, random_state=42)

for train_index, test_val_index in msss.split(X, y):
    X_train, X_test_val = X[train_index], X[test_val_index]
    y_train, y_test_val = y[train_index], y[test_val_index]

msss = MultilabelStratifiedShuffleSplit(n_splits=2, test_size=0.5, random_state=42)

for test_index, val_index in msss.split(X_test_val, y_test_val):
    X_test, X_val = X_test_val[test_index], X_test_val[val_index]
    y_test, y_val = y_test_val[test_index], y_test_val[val_index]


X_train_right = df.loc[df['seat']=='right']['filenames'].to_numpy()
y_train_right = df.loc[df['seat']=='right'][all_features].to_numpy()

X_train = np.concatenate([X_train,X_train_right])
y_train = np.concatenate([y_train,y_train_right])



In [310]:
test_index_real = df_with_right.index[test_index]
val_index_real = df_with_right.index[val_index]


# Create a new column "dataset"
df['dataset'] = 'train'  # Set default value to 'train'

# Assign 'test' for rows in test_index
df.loc[test_index_real, 'dataset'] = 'test'

# Assign 'val' for rows in val_index
df.loc[val_index_real, 'dataset'] = 'val'

distribution = df.groupby(['dataset', 'participant_id_seat']).size().reset_index(name='count')
distribution.set_index(['dataset', 'participant_id_seat']).unstack('dataset').fillna(0)

count               
dataset               test   train    val
participant_id_seat                      
PIS_ID_00_left       576.0     0.0  538.0
PIS_ID_00_right        0.0  1114.0    0.0
PIS_ID_02_left       657.0     0.0  659.0
PIS_ID_02_right        0.0  1316.0    0.0
PIS_ID_04_left       801.0   190.0  837.0
PIS_ID_04_right        0.0  1828.0    0.0
PIS_ID_05_left         0.0  1552.0    0.0
PIS_ID_05_right        0.0  1552.0    0.0
PIS_ID_06_left         0.0   703.0    0.0
PIS_ID_06_right        0.0   703.0    0.0
PIS_ID_07_left         0.0  1622.0    0.0
PIS_ID_07_right        0.0  1622.0    0.0

In [313]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assuming y_train and y_test are binary/multilabel matrices
# and feature_names is a list of class names in the same order

# Sum the labels for each class to get the distribution
# train_class_distribution = y_train[:,:len(feature_names)].sum(axis=0)
# test_class_distribution = y_test[:,:len(feature_names)].sum(axis=0)
# val_class_distribution  = y_val[:,:len(feature_names)].sum(axis=0)

train_class_distribution = y_train[:,:len(feature_names)].sum(axis=0)
test_class_distribution = y_test[:,:len(feature_names)].sum(axis=0)
val_class_distribution  = y_val[:,:len(feature_names)].sum(axis=0)


# Create a DataFrame for easy labeling
df_train = pd.DataFrame({'Class': feature_names, 'Distribution': train_class_distribution})
df_test = pd.DataFrame({'Class': feature_names, 'Distribution': test_class_distribution})
df_val = pd.DataFrame({'Class': feature_names, 'Distribution': val_class_distribution})

df_train['dataset'] = 'train'
df_test['dataset'] = 'test'
df_val['dataset'] = 'val'

df_dataset = pd.concat([df_train, df_val, df_test])
df_dataset = df_dataset.set_index(['Class','dataset']).unstack('dataset').droplevel(0, axis=1)
df_dataset.columns.name = None
df_dataset = df_dataset.reset_index()
df_dataset = df_dataset[['Class','train','val','test']]
df_dataset = df_dataset.sort_values('train', ascending=False)
df_dataset



,Class,train,val,test
0,bg,9676,779,779
10,mov,2526,1255,1255
4,hmov,1363.0,682.0,681.0
3,fmov,767.0,378.0,389.0
6,hrep,722.0,362.0,361.0
5,hnod,603.0,302.0,302.0
7,hshk,274.0,138.0,137.0
8,htfa,242.0,122.0,121.0
11,readj,226.0,114.0,113.0
2,flean,110.0,55.0,55.0


In [314]:
1363.0	+682.0+681.0

2726.0

In [315]:
train_class_distribution = y_train[:,len(feature_names):].sum(axis=0)
test_class_distribution = y_test[:,len(feature_names):].sum(axis=0)
val_class_distribution  = y_val[:,len(feature_names):].sum(axis=0)


In [316]:
def get_class_weights(df,feature_names, alpha=10, beta=2):
    class_weights = {}
    positive_weights = {}
    negative_weights = {}
    # N = len(df)
    n_features = len(feature_names)

    N = np.sum(df[feature_names].to_numpy())
    for label in feature_names:
        if label in df.columns:
            positive_weights[label] = np.log((N+n_features) /(alpha * (sum(df[label] == 1))+1)*beta)
            negative_weights[label] = np.log((N+n_features) /(alpha * (sum(df[label] == 0))+1)*beta)
        else:
            positive_weights[label] = 0
            negative_weights[label] = 0


            
    # class_weights['positive_weights'] = pd.DataFrame.from_dict(positive_weights)
    # class_weights['negative_weights'] = pd.DataFrame.from_dict(negative_weights)
    class_weights = pd.DataFrame(zip(positive_weights.keys(),positive_weights.values(), negative_weights.values()),columns=['class','positive_weights','negative_weights'])        
    class_weights['method'] = 'inv'
    return class_weights

# def get_class_weights_log(df,feature_names, mu=0.15, min_w=1):
#     class_weights = {}
#     positive_weights = {}
#     negative_weights = {}
    
#     n_features = len(feature_names)

#     total = np.sum(df[feature_names].to_numpy())
#     for label in feature_names:
#         if label in df.columns:

#             score = np.log((mu*total+n_features)/float(sum(df[label] == 1)+1))
#             positive_weights[label] = score if score > min_w else min_w

#             score = np.log((mu*total+n_features)/float(sum(df[label] == 0)+1))
#             negative_weights[label] = score if score > min_w else min_w
            

#         else:
#             positive_weights[label] = 0
#             negative_weights[label] = 0


#     class_weights = pd.DataFrame(zip(positive_weights.keys(),positive_weights.values(), negative_weights.values()),columns=['class','positive_weights','negative_weights'])        
#     class_weights['method'] = 'log'
#     # class_weights['positive_weights'] = pd.DataFrame.from_dict(positive_weights)
#     # class_weights['negative_weights'] = pd.DataFrame.from_dict(negative_weights)
#     return class_weights





In [317]:
# mu = 1
# n_features = 13
# s = 10
# total = np.sum(df[feature_names].to_numpy())
# np.log((mu*total+n_features)/float(s+1))

In [318]:

# class_weights_log = get_class_weights_log(df,feature_names, mu=0.5, min_w=1)
class_weights = get_class_weights(df,feature_names, 2, 1)
class_weights = class_weights.drop('method',axis=1)

# df_weights = pd.concat([class_weights, class_weights_log]).set_index(['class','method']).unstack('method')
# df_weights.sort_values(('positive_weights','inv'), ascending=False)
class_weights

,class,positive_weights,negative_weights
0,blean,4.123005,-0.235978
1,flean,4.052554,-0.235044
2,fmov,2.112491,-0.149631
3,hmov,1.537677,-0.065284
4,hnod,2.352144,-0.171579
5,hrep,2.172241,-0.155653
6,hshk,3.139443,-0.214333
7,htfa,3.263272,-0.218396
8,hthd,4.799264,-0.242245
9,readj,3.331456,-0.220421


In [324]:
len(class_weights)

13

In [ ]:
X_train
y_train

X_test
X_val
y_test
y_val

In [319]:
df_train = pd.DataFrame(zip(X_train, y_train), columns=['filenames', 'labels'])
df_test = pd.DataFrame(zip(X_test, y_test), columns=['filenames', 'labels'])
df_val = pd.DataFrame(zip(X_val, y_val), columns=['filenames', 'labels'])




In [320]:
# Create a list for the "dataset" column and set all values to 'train'
dataset_column = ['train'] * len(df)

# Set values in 'test_index' to 'test'
for index in test_index:
    dataset_column[index] = 'test'

# Set values in 'val_index' to 'val'
for index in val_index:
    dataset_column[index] = 'val'

# Assign the "dataset" column to the DataFrame
df['dataset'] = dataset_column

df.set_index('dataset', append=True, inplace=True)
# Display the modified DataFrame
df.head()

,,participant_id,begin,seat,filenames,view,blean,flean,fmov,hmov,hnod,hrep,hshk,htfa,hthd,readj,shrai,mov,bg,participant_id_seat
,dataset,,,,,,,,,,,,,,,,,,,
0,val,PIS_ID_00,0,left,/videos/mpi_data/2Itzik/dyadic_communication/S...,top,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,False,PIS_ID_00_left
1,test,PIS_ID_00,0,left,/videos/mpi_data/2Itzik/dyadic_communication/S...,front,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,False,PIS_ID_00_left
2,test,PIS_ID_00,0,right,/videos/mpi_data/2Itzik/dyadic_communication/S...,top,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,PIS_ID_00_right
3,val,PIS_ID_00,0,right,/videos/mpi_data/2Itzik/dyadic_communication/S...,front,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,PIS_ID_00_right
4,test,PIS_ID_00,2000,left,/videos/mpi_data/2Itzik/dyadic_communication/S...,top,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,True,False,PIS_ID_00_left


In [321]:
save_folder = '/home/ubuntu/efs/videoMAE/scripts/dyadic_communication_5vid_clean_2/dataset'

os.makedirs(save_folder, exist_ok=True)

df_train.to_csv(osp.join(save_folder, 'train.csv'))
df_test.to_csv(osp.join(save_folder, 'test.csv'))
df_val.to_csv(osp.join(save_folder, 'val.csv'))
class_weights.to_csv(osp.join(save_folder, 'weights.csv'))


df.to_csv(osp.join(save_folder, 'all_data.csv'))

In [322]:
import json
wrapped_data = {'data': class_weights.to_dict(orient='records')}

# Save wrapped data as JSON
with open(osp.join(save_folder, 'weights.json'), 'w') as json_file:
    json.dump(wrapped_data, json_file, indent=4)